In [2]:
%matplotlib inline

from __future__ import division # 1/2 = 0.5 instead of 0
from IPython.core.display import *

In [3]:
from unification import *

** Function is_same_state() is used to check whether two states are same or not. **

In [4]:
def is_same_state(state1, state2):
    if len(state1) != len(state2):
        return False
    for i in range(len(state1)):
        if state1[i] != state2[i]:
            return False
    return True

** Function forward_planner() returns a plan from start state to goal state. It uses DFS to explore states and apply actions until it find goal state. **

In [5]:
def forward_planner(start, goal, actions, debug=False):
    state = start[:]
    stack = []
    checked = []
    plan = []
    stack.append(state)
    while stack:
        current_state = stack.pop(0)
        if is_same_state(current_state, goal):
            return plan
        checked.append(current_state)
        for a in actions:
            action = actions[a]["action"]
            conditions = actions[a]["conditions"]
            add = actions[a]["add"][0]
            delete = actions[a]["delete"][0]
            d = {}
            for exp1 in conditions:
                for exp2 in current_state:
                    exp = unify(exp1, exp2)
                    if exp != False:
                        for key in exp:
                            if not d.get(key):
                                d[key] = exp[key]
            for k in d:
                action = action.replace(k, d[k])
                add = add.replace(k, d[k])
                delete = delete.replace(k, d[k])
            next_state = current_state[:]
            for i in range(len(next_state)):
                if next_state[i] == delete:
                    next_state[i] = add
            if next_state in checked:
                continue
            stack.append(next_state)
            plan.append(action)
    return []

In [6]:
start_state = [
    "(item Drill)",
    "(place1 Home)",
    "(place2 Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Store)"
]

In [7]:
goal_state = [
    "(item Drill)",
    "(place1 Home)",
    "(place2 Store)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)"
]

In [8]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place1 ?from)",
            "(place2 ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place2 ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

In [9]:
plan = forward_planner(start_state, goal_state, actions)
while plan:
    print plan.pop()

(drive Me Home Store)
(buy Me Store Drill)
